In [1]:
import os 
import pandas as pd

In [5]:
data_dir = "/workspaces/Recommenders-Systems/data"
books_path = os.path.join(data_dir,"merged_df.pkl")

In [7]:
books_df= pd.read_pickle(books_path)
books_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location,Age,Rating_Count
0,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,"seattle, washington, usa",32,1
1,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,"rijeka, n/a, croatia",16,1
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,"rijeka, n/a, croatia",16,1
3,276744,038550120X,7,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,"torrance, california, usa",32,81
4,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,http://images.amazon.com/images/P/0060517794.0...,"iowa city, iowa, usa",25,30


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
def load_data_in_chunks(file_path, chunk_size=100000):
    for chunk in pd.read_pickle(file_path, chunksize=chunk_size):
        yield chunk

In [ ]:
books_path

In [32]:
def preprocess_data(data):
    le_user = LabelEncoder()
    le_isbn = LabelEncoder()
    
    data['User-ID'] = le_user.fit_transform(data['User-ID'])
    data['ISBN'] = le_isbn.fit_transform(data['ISBN'])
    
    return data, le_user, le_isbn

# Création de la matrice creuse
def create_sparse_matrix(data):
    rows = data['User-ID'].values
    cols = data['ISBN'].values
    values = data['Book-Rating'].values
    
    return csr_matrix((values, (rows, cols)))

# Modèle SVD
def train_svd(matrix, n_components=50):
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    return svd.fit(matrix)

# Modèle KNN
def train_knn(matrix, n_neighbors=20):
    knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_neighbors, n_jobs=-1)
    return knn.fit(matrix)

# Fonction d'évaluation
def evaluate_model(predictions, actual):
    return np.sqrt(mean_squared_error(actual, predictions))

# Fonction de recommandation
def get_top_n_recommendations(model, user_vector, n=10):
    if isinstance(model, TruncatedSVD):
        user_ratings = model.transform(user_vector).dot(model.components_)
    elif isinstance(model, NearestNeighbors):
        _, indices = model.kneighbors(user_vector)
        user_ratings = matrix[indices.flatten()].mean(axis=0).A1
    
    top_n_indices = np.argsort(user_ratings)[-n:][::-1]
    return top_n_indices